In [1]:
import json
from p4util_builder.fattree import TopologyBuilder, FatTreeTopology

In [31]:
def generateL2Graph(start:str, switches:list, hosts:list, info:dict):
    '''
    generator use and it shouldn't be called directly
    '''
    que = [(start, -1)]
    vis = {}
    res = {}
    vis[start] = 1
    while(len(que) != 0):
        u, sp = que.pop(0)
        for nxt in info[u]["interfaces_to_node"]:
            nxt_port = info[u]["interfaces_to_port"][nxt]
            nxt_dev = info[u]["interfaces_to_node"][nxt]
            if nxt_dev in vis:
                continue
            vis[nxt_dev] = 1
            if nxt_dev in hosts:
                res[nxt_dev] = {
                    "mac": info[nxt_dev][u]["mac"],
                    "port": (sp == -1 ? nxt_port : sp)
                }
                continue
            elif nxt_dev in switches:
                que.push((nxt_dev, (sp == -1 ? nxt_port : sp)))
    return res
    
def generateL2Table(topo:TopologyBuilder, topo_db:str):
    '''
    Here topo can be any type extends from Topology Builder
    topo_db is ".db" file generated when creating topo with the configuration file created by TopologyBUilder
    and the ".db" file is in the directory where you type `sudo p4run --config xxx.json`
    
    And your p4 table name should be l2f, forward function should be defined like
    ```
    action l2forward(egressPort_t port) {
        ...
    }
    ```
    '''
    import copy
    new_topo = copy.deepcopy(topo)
    info = None
    with open(topo_db, 'r') as f:
        info = json.load(f)
    switch_names = list([s.name for s in topo.switches])
    host_names = list([h.name for h in topo.hosts])
    dir_dict = {}
    for k in info:
        if k in switch_names:
            switch = info[k]
            for nxt in switch["interfaces_to_node"]:
                dir_dict[(k, switch["interfaces_to_node"][nxt])] = switch["interfaces_to_port"][nxt]
    result = ""
    for k in switch_names:
        n2pdic = generateL2Graph(k, switch_names, host_names, info)
        result += f"echo > {k}-commands.txt << END\n"
        for h in n2pdic:
            result += f"table_add l2f l2forward {n2pdic[h]['mac']} => {n2pdic[h]['port']}\n"
        result += "END\n"
        for s in new_topo.switches:
            if s.name == k:
                s.setP4CLI(f"{k}-commands.txt")
                break
    return (new_topo, result)

In [36]:
test = FatTreeTopology()
test.populateP4("test.p4")
result = generateL2Table(test, "topo_test.db")

print(result[1])
print(result[0].generate())

echo > sm_10-commands.txt << END
END
echo > sm_11-commands.txt << END
END
echo > sm_12-commands.txt << END
END
echo > sm_13-commands.txt << END
END
echo > sm_1000-commands.txt << END
END
echo > sm_1001-commands.txt << END
END
echo > sm_1010-commands.txt << END
END
echo > sm_1011-commands.txt << END
END
echo > sm_1100-commands.txt << END
END
echo > sm_1101-commands.txt << END
END
echo > sm_1110-commands.txt << END
END
echo > sm_1111-commands.txt << END
END
echo > sm_1200-commands.txt << END
END
echo > sm_1201-commands.txt << END
END
echo > sm_1210-commands.txt << END
END
echo > sm_1211-commands.txt << END
END
echo > sm_1300-commands.txt << END
END
echo > sm_1301-commands.txt << END
END
echo > sm_1310-commands.txt << END
END
echo > sm_1311-commands.txt << END
END

{
    "program": "default.p4",
    "switch": "simple_switch",
    "compiler": "p4c",
    "options": "--target bmv2 --arch v1model --std p4-16",
    "switch_cli": "simple_switch_CLI",
    "cli": true,
    "pcap_dump": true,
    